DATA GATHERING

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Target URL
url = "https://dps.psx.com.pk/market-watch"

# Send a GET request
response = requests.get(url)
print(response.status_code)
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify())

200
<table class="tbl" data-page-length="25">
 <thead class="tbl__head">
  <!-- tr-->
  <!--   th(colspan="11")-->
  <!--   th.center(colspan="2", style="width: 110px;") TOTAL BUY ORDERS-->
  <!--   th.center(colspan="2", style="width: 110px;") TOTAL SELL ORDERS-->
  <tr>
   <th data-name="symbol">
    SYMBOL
   </th>
   <th data-name="sector">
    SECTOR
   </th>
   <th data-name="listed">
    LISTED IN
   </th>
   <th class="right" data-name="ldcp">
    LDCP
   </th>
   <th class="right" data-name="open">
    OPEN
   </th>
   <th class="right" data-name="high">
    HIGH
   </th>
   <th class="right" data-name="low">
    LOW
   </th>
   <th class="right" data-name="close">
    CURRENT
   </th>
   <th class="right" data-name="change">
    CHANGE
   </th>
   <th class="right" data-name="percentChange">
    CHANGE (%)
   </th>
   <th class="right" data-name="volume">
    VOLUME
   </th>
   <!-- th.right(data-name="obq") QUEUED*-->
   <!-- th.right(data-name="obc") CANCELLED**-->
   <!-- 

In [13]:
companies=soup.find_all('tr')
data = []
for c in companies:
    cols = c.find_all('td')
    if len(cols) >= 3:
        a_tag = cols[0].find('a')
        symbol = a_tag.find('strong').text.strip() if a_tag else None
        href = f"https://dps.psx.com.pk{a_tag['href']}" if a_tag else None
        title = a_tag.get('data-title') if a_tag else None
        sector_id = cols[1].text.strip()
        listed_in = cols[2].text.strip()

        data.append({
            'symbol': symbol,
            'href': href,
            'name': title,
            'sector_id': sector_id,
            'listed_in': listed_in
        })
# print(data)
print(len(data))
df = pd.DataFrame(data)
# print(df)
# Mapping sector_id to sector_name based on the provided dropdown options
sector_mapping = {
    "0801": "AUTOMOBILE ASSEMBLER",
    "0802": "AUTOMOBILE PARTS & ACCESSORIES",
    "0803": "CABLE & ELECTRICAL GOODS",
    "0804": "CEMENT",
    "0805": "CHEMICAL",
    "0806": "CLOSE - END MUTUAL FUND",
    "0807": "COMMERCIAL BANKS",
    "0808": "ENGINEERING",
    "0809": "FERTILIZER",
    "0810": "FOOD & PERSONAL CARE PRODUCTS",
    "0811": "GLASS & CERAMICS",
    "0812": "INSURANCE",
    "0813": "INV. BANKS / INV. COS. / SECURITIES COS.",
    "0814": "JUTE",
    "0815": "LEASING COMPANIES",
    "0816": "LEATHER & TANNERIES",
    "0818": "MISCELLANEOUS",
    "0819": "MODARABAS",
    "0820": "OIL & GAS EXPLORATION COMPANIES",
    "0821": "OIL & GAS MARKETING COMPANIES",
    "0822": "PAPER, BOARD & PACKAGING",
    "0823": "PHARMACEUTICALS",
    "0824": "POWER GENERATION & DISTRIBUTION",
    "0825": "REFINERY",
    "0826": "SUGAR & ALLIED INDUSTRIES",
    "0827": "SYNTHETIC & RAYON",
    "0828": "TECHNOLOGY & COMMUNICATION",
    "0829": "TEXTILE COMPOSITE",
    "0830": "TEXTILE SPINNING",
    "0831": "TEXTILE WEAVING",
    "0832": "TOBACCO",
    "0833": "TRANSPORT",
    "0834": "VANASPATI & ALLIED INDUSTRIES",
    "0835": "WOOLLEN",
    "0836": "REAL ESTATE INVESTMENT TRUST",
    "0837": "EXCHANGE TRADED FUNDS",
    "0838": "PROPERTY"
}

# Map sector_id to sector_name
df['sector_name'] = df['sector_id'].map(sector_mapping)


441


In [14]:
print(df.head())

   symbol                                   href  \
0     WTL     https://dps.psx.com.pk/company/WTL   
1  CNERGY  https://dps.psx.com.pk/company/CNERGY   
2    SSGC    https://dps.psx.com.pk/company/SSGC   
3     BOP     https://dps.psx.com.pk/company/BOP   
4     PRL     https://dps.psx.com.pk/company/PRL   

                               name sector_id                listed_in  \
0         Worldcall Telecom Limited      0828                   ALLSHR   
1             Cnergyico PK  Limited      0825  ALLSHR,KMIALLSHR,KSE100   
2  Sui Southern Gas Company Limited      0821   ALLSHR,JSMFI,KMIALLSHR   
3                The Bank of Punjab      0807            ALLSHR,KSE100   
4         Pakistan Refinery Limited      0825   ALLSHR,KMI30,KMIALLSHR   

                     sector_name  
0     TECHNOLOGY & COMMUNICATION  
1                       REFINERY  
2  OIL & GAS MARKETING COMPANIES  
3               COMMERCIAL BANKS  
4                       REFINERY  


In [15]:
import pandas as pd

# Let's assume your DataFrame is named df
# First, split the 'listed_in' column by comma
df['listed_in_split'] = df['listed_in'].str.split(',')

# Use explode + get_dummies to one-hot encode
one_hot = df.explode('listed_in_split')
one_hot_encoded = pd.get_dummies(one_hot['listed_in_split'])

# Group back by index to restore original rows
one_hot_encoded = one_hot_encoded.groupby(one_hot.index).sum()

# Drop intermediate column and join
df = df.drop(columns='listed_in_split').join(one_hot_encoded)

print(df.head())


   symbol                                   href  \
0     WTL     https://dps.psx.com.pk/company/WTL   
1  CNERGY  https://dps.psx.com.pk/company/CNERGY   
2    SSGC    https://dps.psx.com.pk/company/SSGC   
3     BOP     https://dps.psx.com.pk/company/BOP   
4     PRL     https://dps.psx.com.pk/company/PRL   

                               name sector_id                listed_in  \
0         Worldcall Telecom Limited      0828                   ALLSHR   
1             Cnergyico PK  Limited      0825  ALLSHR,KMIALLSHR,KSE100   
2  Sui Southern Gas Company Limited      0821   ALLSHR,JSMFI,KMIALLSHR   
3                The Bank of Punjab      0807            ALLSHR,KSE100   
4         Pakistan Refinery Limited      0825   ALLSHR,KMI30,KMIALLSHR   

                     sector_name     ACI  ALLSHR  BKTI  ...  KMIALLSHR  \
0     TECHNOLOGY & COMMUNICATION  0    0       1     0  ...          0   
1                       REFINERY  0    0       1     0  ...          1   
2  OIL & GAS MARKETI

In [17]:
df['data-link'] = "https://dps.psx.com.pk/timeseries/eod/" + df['symbol']
print(df.head())

   symbol                                   href  \
0     WTL     https://dps.psx.com.pk/company/WTL   
1  CNERGY  https://dps.psx.com.pk/company/CNERGY   
2    SSGC    https://dps.psx.com.pk/company/SSGC   
3     BOP     https://dps.psx.com.pk/company/BOP   
4     PRL     https://dps.psx.com.pk/company/PRL   

                               name sector_id                listed_in  \
0         Worldcall Telecom Limited      0828                   ALLSHR   
1             Cnergyico PK  Limited      0825  ALLSHR,KMIALLSHR,KSE100   
2  Sui Southern Gas Company Limited      0821   ALLSHR,JSMFI,KMIALLSHR   
3                The Bank of Punjab      0807            ALLSHR,KSE100   
4         Pakistan Refinery Limited      0825   ALLSHR,KMI30,KMIALLSHR   

                     sector_name     ACI  ALLSHR  BKTI  ...  KSE100  KSE30  \
0     TECHNOLOGY & COMMUNICATION  0    0       1     0  ...       0      0   
1                       REFINERY  0    0       1     0  ...       1      0   
2  OIL &

In [18]:
df.to_csv('company_data.csv', index=False)

In [19]:
import requests
import pandas as pd
import time

all_5y_data = []  # To collect all companies' timeseries

for symbol in df['symbol']:
    url = f"https://dps.psx.com.pk/timeseries/eod/{symbol}"
    try:
        response = requests.get(url)
        data = response.json()

        if data["status"] == 1 and "data" in data:
            company_data = data["data"]

            # Create DataFrame
            company_df = pd.DataFrame(company_data, columns=["timestamp", "close_price", "volume", "open_price"])
            company_df["timestamp"] = pd.to_datetime(company_df["timestamp"], unit="s")
            company_df["symbol"] = symbol
            print(company_df.head())
            print("/n")

            all_5y_data.append(company_df)

            # Optional: Save individual CSVs
            # company_df.to_csv(f"{symbol}_timeseries.csv", index=False)

        else:
            print(f"No data for {symbol}")

    except Exception as e:
        print(f"Error fetching {symbol}: {e}")

    time.sleep(1)  # Polite delay to avoid hitting server too fast

# Combine all into one DataFrame (optional)
combined_df = pd.concat(all_5y_data, ignore_index=True)

# Save combined CSV
# combined_df.to_csv("all_companies_timeseries.csv", index=False)


            timestamp  close_price    volume  open_price symbol
0 2025-05-08 11:00:00         1.10  93253525        1.25    WTL
1 2025-05-07 11:00:00         1.22  53673828        1.20    WTL
2 2025-05-06 11:00:00         1.33  10597193        1.35    WTL
3 2025-05-05 11:00:00         1.34  23846954        1.29    WTL
4 2025-05-02 11:00:00         1.27  11631669        1.27    WTL
/n
            timestamp  close_price    volume  open_price  symbol
0 2025-05-08 11:00:00         5.72  19599374        6.76  CNERGY
1 2025-05-07 11:00:00         6.58  15756609        6.41  CNERGY
2 2025-05-06 11:00:00         7.09   8030127        7.14  CNERGY
3 2025-05-05 11:00:00         7.05   8200468        7.15  CNERGY
4 2025-05-02 11:00:00         7.11  27367683        7.01  CNERGY
/n
            timestamp  close_price    volume  open_price symbol
0 2025-05-08 11:00:00        29.22  15067745       31.99   SSGC
1 2025-05-07 11:00:00        32.47  26711578       33.30   SSGC
2 2025-05-06 11:00:00       

In [20]:
# Save combined CSV
combined_df.to_csv("all_companies_timeseries.csv", index=False)

In [1]:
print(combined_df.head())
print(df.head())

NameError: name 'combined_df' is not defined

In [3]:
import pandas as pd

company_metadata = pd.read_csv('company_data.csv')
print(company_metadata.head())

   symbol                                   href  \
0     WTL     https://dps.psx.com.pk/company/WTL   
1  CNERGY  https://dps.psx.com.pk/company/CNERGY   
2    SSGC    https://dps.psx.com.pk/company/SSGC   
3     BOP     https://dps.psx.com.pk/company/BOP   
4     PRL     https://dps.psx.com.pk/company/PRL   

                               name  sector_id                listed_in  \
0         Worldcall Telecom Limited        828                   ALLSHR   
1             Cnergyico PK  Limited        825  ALLSHR,KMIALLSHR,KSE100   
2  Sui Southern Gas Company Limited        821   ALLSHR,JSMFI,KMIALLSHR   
3                The Bank of Punjab        807            ALLSHR,KSE100   
4         Pakistan Refinery Limited        825   ALLSHR,KMI30,KMIALLSHR   

                     sector_name  Unnamed: 6  ACI  ALLSHR  BKTI  ...  KSE100  \
0     TECHNOLOGY & COMMUNICATION           0    0       1     0  ...       0   
1                       REFINERY           0    0       1     0  ...       1

In [11]:
import pandas as pd

# Load both CSVs
company_metadata = pd.read_csv('company_data.csv')
company_timeseries = pd.read_csv('all_companies_timeseries.csv')

# Convert timestamp to datetime and sort
company_timeseries['timestamp'] = pd.to_datetime(company_timeseries['timestamp'])
company_timeseries_sorted = company_timeseries.sort_values(by='timestamp')

# Select only required columns from metadata
metadata_subset = company_metadata[['symbol', 'sector_id', 'sector_name']]

# Merge on the 'symbol' column
merged_df = pd.merge(company_timeseries_sorted, metadata_subset, on='symbol', how='left')

# View the merged result
print(merged_df.head())


            timestamp  close_price  volume  open_price symbol  sector_id  \
0 2020-05-11 09:01:19       526.66     550      525.00   ARPL        805   
1 2020-05-11 09:01:30        27.02  589500       27.15   EPCL        805   
2 2020-05-11 09:01:44        21.77  750000       21.67  KAPCO        824   
3 2020-05-11 09:01:46        61.50    1000       61.50  JSGCL        813   
4 2020-05-11 09:01:47       290.00     200      290.00  JLICL        812   

                                sector_name  
0                                  CHEMICAL  
1                                  CHEMICAL  
2           POWER GENERATION & DISTRIBUTION  
3  INV. BANKS / INV. COS. / SECURITIES COS.  
4                                 INSURANCE  


In [12]:
# Save merged DataFrame to CSV
merged_df.to_csv('sectorwise_company_timeseries.csv', index=False)

PREPROCESSING + TRAINING

In [26]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Load the dataset
df = pd.read_csv("sectorwise_company_timeseries.csv", parse_dates=['timestamp'])

# Sort and group
df.sort_values(['symbol', 'timestamp'], inplace=True)
print(df.head())
df['sector_name'] = df['sector_name'].replace('INV. BANKS / INV. COS. / SECURITIES COS.', 'BANKS')
print(df.head())

               timestamp  close_price  volume  open_price symbol  sector_id  \
114  2020-05-11 09:08:02        38.74  567500       39.00    786        813   
374  2020-05-12 09:07:50        38.72  598000       38.51    786        813   
662  2020-05-13 09:08:17        39.32  803000       38.55    786        813   
936  2020-05-14 09:09:11        38.94  732500       39.51    786        813   
1339 2020-05-15 08:38:46        38.77  741000       38.61    786        813   

                                   sector_name  
114   INV. BANKS / INV. COS. / SECURITIES COS.  
374   INV. BANKS / INV. COS. / SECURITIES COS.  
662   INV. BANKS / INV. COS. / SECURITIES COS.  
936   INV. BANKS / INV. COS. / SECURITIES COS.  
1339  INV. BANKS / INV. COS. / SECURITIES COS.  
               timestamp  close_price  volume  open_price symbol  sector_id  \
114  2020-05-11 09:08:02        38.74  567500       39.00    786        813   
374  2020-05-12 09:07:50        38.72  598000       38.51    786        8

In [27]:
#sector wise seperation to train models specific to sectors
def create_multioutput_dataset(dataset, time_step=100, horizons=[1, 5, 22, 132]):
    X, y_all = [], []
    for i in range(len(dataset) - time_step - max(horizons)):
        X.append(dataset[i:i+time_step])
        y = [dataset[i+time_step+h-1, 0] for h in horizons]  # 0 = close_price
        y_all.append(y)
    return np.array(X), np.array(y_all)


In [28]:
from sklearn.preprocessing import MinMaxScaler

time_step = 100
horizons = [1, 5, 22, 132]  # 1 day, 1 week, 1 month, 6 months
sectors = df['sector_name'].unique()
sector_datasets = {}

for sector in sectors:
    sector_df = df[df['sector_name'] == sector]
    companies = sector_df['symbol'].unique()

    X_all, y_all = [], []
    for symbol in companies:
        company_df = sector_df[sector_df['symbol'] == symbol]
        company_df = company_df[['close_price', 'volume', 'open_price']].values

        scaler = MinMaxScaler()
        company_scaled = scaler.fit_transform(company_df)

        X, y = create_multioutput_dataset(company_scaled, time_step, horizons)

        if len(X) > 0 and len(y) > 0:
            X_all.append(X)
            y_all.append(y)

    if X_all and y_all:
        X_all = np.concatenate(X_all, axis=0)
        y_all = np.concatenate(y_all, axis=0)
        sector_datasets[sector] = (X_all, y_all)


In [30]:
print(len(sector_datasets))
print(sector_datasets.keys())
print(sector_datasets["CHEMICAL"])
for key in sector_datasets:
    X, y = sector_datasets[key]
    print(f"{key}: X.shape = {X.shape}, y.shape = {y.shape}")


36
dict_keys(['BANKS', 'SUGAR & ALLIED INDUSTRIES', 'TEXTILE SPINNING', 'COMMERCIAL BANKS', 'PHARMACEUTICALS', 'EXCHANGE TRADED FUNDS', 'CEMENT', 'TEXTILE COMPOSITE', 'ENGINEERING', 'INSURANCE', 'AUTOMOBILE PARTS & ACCESSORIES', 'FERTILIZER', 'AUTOMOBILE ASSEMBLER', 'TECHNOLOGY & COMMUNICATION', 'MISCELLANEOUS', 'POWER GENERATION & DISTRIBUTION', 'OIL & GAS MARKETING COMPANIES', 'CHEMICAL', 'FOOD & PERSONAL CARE PRODUCTS', 'REFINERY', 'LEATHER & TANNERIES', 'MODARABAS', 'GLASS & CERAMICS', 'WOOLLEN', 'PROPERTY', 'PAPER, BOARD & PACKAGING', 'TRANSPORT', 'REAL ESTATE INVESTMENT TRUST', 'CABLE & ELECTRICAL GOODS', 'SYNTHETIC & RAYON', 'CLOSE - END MUTUAL FUND', 'TEXTILE WEAVING', 'TOBACCO', 'OIL & GAS EXPLORATION COMPANIES', 'LEASING COMPANIES', 'VANASPATI & ALLIED INDUSTRIES'])
(array([[[6.63601670e-01, 2.35484028e-03, 7.93051360e-01],
        [6.84962733e-01, 3.02004940e-03, 8.00604230e-01],
        [6.84962733e-01, 7.01130412e-03, 8.00604230e-01],
        ...,
        [7.38006813e-01, 

In [31]:
from sklearn.model_selection import train_test_split

sector_train_datasets = {}
sector_test_datasets = {}

for sector, (X, y) in sector_datasets.items():
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    sector_train_datasets[sector] = (X_train, y_train)
    sector_test_datasets[sector] = (X_test, y_test)


In [32]:
print(len(sector_train_datasets))

36


In [33]:
#model structure

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D

def build_transformer_multioutput(input_shape, output_dim=4):
    inputs = Input(shape=input_shape)
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
    x = Dropout(0.1)(x)
    res = x + inputs

    x = LayerNormalization(epsilon=1e-6)(res)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(input_shape[-1])(x)
    x = x + res

    x = GlobalAveragePooling1D()(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(output_dim)(x)  # 4 outputs

    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mse')
    return model


In [ ]:
#training
for sector, (X, y) in sector_train_datasets.items():
    print(f"Training model for sector: {sector}")
    model = build_transformer_multioutput(input_shape=X.shape[1:])
    model.fit(X, y, epochs=30, batch_size=64, validation_split=0.2, verbose=1)
    model.save(f"/models/model_{sector.replace(' ', '_')}_multioutput.h5")


In [ ]:
#model summaries

import os

# Directory containing the saved model files
model_dir = "models/"  # Change to the actual directory if needed

# Get the list of model files (with .h5 extension)
model_files = [f for f in os.listdir(model_dir) if f.endswith('.h5')]

# Iterate through each model file, rebuild the model, and load the weights
for model_file in model_files:
    sector_name = model_file.replace('model_', '').replace('_multioutput.h5', '').replace('_', ' ')  # Extract sector name

    print(f"Loading model for sector: {sector_name}")

    input_shape = (100, 3)
    model = build_transformer_multioutput(input_shape=input_shape)

    # Load the model weights
    model.load_weights(os.path.join(model_dir, model_file))

    # Print the model summary
    print(f"Model summary for {sector_name}:")
    model.summary()
    print("\n" + "="*50 + "\n")

TESTING ON SECTORS

In [41]:
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

results = {}

model_dir = "models/"  # Change to the actual directory if needed

# Get the list of model files (with .h5 extension)
model_files = [f for f in os.listdir(model_dir) if f.endswith('.h5')]

for sector, (X_test, y_test) in sector_test_datasets.items():
    print(f"Testing model for sector: {sector}")

    input_shape = (100, 3)
    model = build_transformer_multioutput(input_shape=input_shape)

    # Load the model weights
    model.load_weights(os.path.join(model_dir, model_file))

    # Predict
    y_pred = model.predict(X_test)

    # Evaluate
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"Sector: {sector} | MSE: {mse:.4f} | R²: {r2:.4f}")
    results[sector] = {'mse': mse, 'r2': r2}


Testing model for sector: BANKS
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step
Sector: BANKS | MSE: 0.1771 | R²: -2.9821
Testing model for sector: SUGAR & ALLIED INDUSTRIES
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
Sector: SUGAR & ALLIED INDUSTRIES | MSE: 0.0483 | R²: -0.0052
Testing model for sector: TEXTILE SPINNING
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step
Sector: TEXTILE SPINNING | MSE: 0.0749 | R²: -0.3117
Testing model for sector: COMMERCIAL BANKS
115/115 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step
Sector: COMMERCIAL BANKS | MSE: 0.2211 | R²: -4.2232
Testing model for sector: PHARMACEUTICALS
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
Sector: PHARMACEUTICALS | MSE: 0.2086 | R²: -2.7563
Testing model for sector: EXCHANGE TRADED FUNDS
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Sector: EXCHANGE TRADED FUNDS | MSE: 0.1313 | R²: -0.8075
Testing model for sector: CEMENT
104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
Sector: CEMENT | MSE: 0.1826 | R²: -4.2060
Testing model for sector: TEXTILE COMPOSITE
119/119 ━━━━

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Sector: PROPERTY | MSE: 0.1828 | R²: -3.7430
Testing model for sector: PAPER, BOARD & PACKAGING
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
Sector: PAPER, BOARD & PACKAGING | MSE: 0.0731 | R²: -0.3130
Testing model for sector: TRANSPORT
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
Sector: TRANSPORT | MSE: 0.1918 | R²: -1.9714
Testing model for sector: REAL ESTATE INVESTMENT TRUST
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Sector: REAL ESTATE INVESTMENT TRUST | MSE: 0.2941 | R²: -11.1123
Testing model for sector: CABLE & ELECTRICAL GOODS
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
Sector: CABLE & ELECTRICAL GOODS | MSE: 0.1414 | R²: -1.6357
Testing model for sector: SYNTHETIC & RAYON
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Sector: SYNTHETIC & RAYON | MSE: 0.1346 | R²: -1.3089
Testing model for sector: CLOSE - END MUTUAL FUND
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Sector: CLOSE - END MUTUAL FUND | MSE: 0.0479 | R²: -0.2902
Testing model for sector: TEXTILE WEAVING


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Sector: VANASPATI & ALLIED INDUSTRIES | MSE: 0.0390 | R²: -0.4134


In [47]:
import matplotlib.pyplot as plt
import os

# Create the output directory if it doesn't exist
results_dir = 'sector_results'
os.makedirs(results_dir, exist_ok=True)

for sector, (X_test, y_test) in sector_test_datasets.items():
    print(f"Saving predictions for sector: {sector}")

    input_shape = X_test.shape[1:]
    model_file = f"model_{sector.replace(' ', '_')}_multioutput.h5"
    model_path = os.path.join(model_dir, model_file)

    if not os.path.exists(model_path):
        print(f"Model not found for sector: {sector}")
        continue

    # Rebuild and load model
    model = build_transformer_multioutput(input_shape=input_shape)
    model.load_weights(model_path)

    # Predict
    y_pred = model.predict(X_test)

    num_outputs = y_test.shape[1]
    fig, axs = plt.subplots(num_outputs, 1, figsize=(12, 3 * num_outputs), sharex=True)
    fig.suptitle(f'{sector} - Actual vs Predicted (Last 100 Samples)', fontsize=16)

    for i in range(num_outputs):
        axs[i].plot(y_test[-100:, i], label='Actual', marker='o', linestyle='-', alpha=0.7)
        axs[i].plot(y_pred[-100:, i], label='Predicted', marker='x', linestyle='--', alpha=0.7)
        axs[i].set_ylabel(f'Output {i + 1}')
        axs[i].legend()
        axs[i].grid(True)

    axs[-1].set_xlabel('Sample Index')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    # Save the plot
    filename = f"{sector.replace(' ', '_')}_results.png"
    filepath = os.path.join(results_dir, filename)
    plt.savefig(filepath)
    plt.close(fig)  # Close the figure to free memory


Saving predictions for sector: BANKS
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step
Saving predictions for sector: SUGAR & ALLIED INDUSTRIES
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
Saving predictions for sector: TEXTILE SPINNING
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step
Saving predictions for sector: COMMERCIAL BANKS
115/115 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step
Saving predictions for sector: PHARMACEUTICALS
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
Saving predictions for sector: EXCHANGE TRADED FUNDS
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
Saving predictions for sector: CEMENT
104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
Saving predictions for sector: TEXTILE COMPOSITE
119/119 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step
Saving predictions for sector: ENGINEERING
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
Saving predictions for sector: INSURANCE
106/106 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
Saving predictions for sector: AUTOMOBILE PARTS & ACCESSORIES
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
Saving predictions for se

In [48]:
#testing on company data

from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

# Define time step and horizons
time_step = 100
horizons = [1, 5, 22, 132]  # 1 day, 1 week, 1 month, 6 months

# Create a function to build the multi-output dataset
def create_multioutput_dataset(scaled_data, time_step, horizons):
    X, y = [], []

    for i in range(len(scaled_data) - time_step):
        X.append(scaled_data[i:i + time_step])
        y_values = []

        for horizon in horizons:
            if i + time_step + horizon <= len(scaled_data):
                y_values.append(scaled_data[i + time_step + horizon - 1, 0])  # Assuming target is the close_price (index 0)
            else:
                y_values.append(np.nan)

        y.append(y_values)

    X = np.array(X)
    y = np.array(y)

      # Handle NaN rows removal
    if y.ndim == 1:
        valid_indices = ~np.isnan(y)
    else:
        valid_indices = ~np.isnan(y).any(axis=1)

    X = X[valid_indices]
    y = y[valid_indices]

    return X, y

# Assuming df is already loaded as per the earlier steps
sectors = df['sector_name'].unique()
sector_company_test_datasets = {}

# Loop through each sector to create the dataset
for sector in sectors:
    sector_df = df[df['sector_name'] == sector]
    companies = sector_df['symbol'].unique()

    sector_results = {}

    for symbol in companies:
        company_df = sector_df[sector_df['symbol'] == symbol]
        company_df = company_df[['close_price', 'volume', 'open_price']].values

        # Normalize the features
        scaler = MinMaxScaler()
        company_scaled = scaler.fit_transform(company_df)

        # Create X and y using the helper function
        X, y = create_multioutput_dataset(company_scaled, time_step, horizons)

        if len(X) > 0 and len(y) > 0:
            sector_results[symbol] = (X, y)

    if sector_results:
        sector_company_test_datasets[sector] = sector_results

# Output to check the structure
print(f"Created {len(sector_company_test_datasets)} sectors with companies.")
print(f"Sample data for sector 'BANKS' and company 'AHL': {sector_company_test_datasets['BANKS']['AHL'][0].shape} -> {sector_company_test_datasets['BANKS']['AHL'][1].shape}")


Created 36 sectors with companies.
Sample data for sector 'BANKS' and company 'AHL': (996, 100, 3) -> (996, 4)


In [49]:
print(sector_company_test_datasets['BANKS'].keys())

dict_keys(['786', 'AHL', 'AKDSL', 'AMBL', 'CASH', 'CYAN', 'DEL', 'DLL', 'ENGROH', 'ESBL', 'FCIBL', 'FCSC', 'FDPL', 'FNEL', 'ICIBL', 'IMS', 'JSCL', 'JSCLPSA', 'JSGCL', 'JSIL', 'LSECL', 'LSEVL', 'MCBIM', 'NEXT', 'OLPL', 'PASL', 'PIAHCLA', 'PIAHCLB', 'PSX'])


In [50]:
X_test, y_test = sector_company_test_datasets['BANKS']['786']
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("y_test:", y_test)

X_test shape: (806, 100, 3)
y_test shape: (806, 4)
y_test: [[0.23402617 0.20733898 0.22119579 0.11316397]
 [0.23248653 0.19245574 0.20862202 0.12188863]
 [0.22940724 0.2181165  0.20887862 0.12291506]
 ...
 [0.05055171 0.05594047 0.07954837 0.11521683]
 [0.03900436 0.04952528 0.0805748  0.11521683]
 [0.05799333 0.05132153 0.07210675 0.09212215]]


In [51]:
import numpy as np

# Assuming y_test is a numpy array
nan_count = np.isnan(y_test).sum()

print(f"Total NaN values: {nan_count}")

Total NaN values: 0


In [55]:
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf

# Directories
output_dir = 'company_results'
os.makedirs(output_dir, exist_ok=True)
csv_path = os.path.join(output_dir, 'metrics_summary.csv')

# List to hold metric rows
metrics_data = []

# Model directory
model_dir = 'models/'

for sector, companies in sector_company_test_datasets.items():
    print(f"Evaluating and plotting predictions for sector: {sector}")

    for company, (X_test, y_test) in companies.items():
        print(f"Evaluating model for company: {company}")

        input_shape = X_test.shape[1:]
        model_file = f"model_{sector.replace(' ', '_')}_multioutput.h5"
        model_path = os.path.join(model_dir, model_file)

        if not os.path.exists(model_path):
            print(f"Model not found for sector: {sector}, company: {company}")
            continue

        # Load model
        model = build_transformer_multioutput(input_shape=input_shape)
        model.load_weights(model_path)

        # Predict
        y_pred = model.predict(X_test)

        # Calculate metrics
        num_outputs = y_test.shape[1]
        fig, axs = plt.subplots(num_outputs, 1, figsize=(12, 3 * num_outputs), sharex=True)
        fig.suptitle(f'{sector} - {company} - Actual vs Predicted (Last 100 Samples)', fontsize=16)

        for i in range(num_outputs):
            mse = mean_squared_error(y_test[:, i], y_pred[:, i])
            r2 = r2_score(y_test[:, i], y_pred[:, i])

            # Append row to CSV data
            metrics_data.append({
                'Sector': sector,
                'Company': company,
                'Output': f'Output_{i+1}',
                'MSE': round(mse, 6),
                'R2': round(r2, 6)
            })

            # Plotting
            axs[i].plot(y_test[-100:, i], label='Actual', marker='o', linestyle='-', alpha=0.7)
            axs[i].plot(y_pred[-100:, i], label='Predicted', marker='x', linestyle='--', alpha=0.7)
            axs[i].set_ylabel(f'Output {i + 1}')
            axs[i].legend()
            axs[i].grid(True)

        axs[-1].set_xlabel('Sample Index')
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])

        # Save plot
        filename = f"{sector.replace(' ', '_')}_{company}_results.png"
        save_path = os.path.join(output_dir, filename)
        plt.savefig(save_path)
        plt.close()

# Save all metrics to CSV
metrics_df = pd.DataFrame(metrics_data)
metrics_df.to_csv(csv_path, index=False)
print(f"\nAll metrics saved to: {csv_path}")


Evaluating and plotting predictions for sector: BANKS
Evaluating model for company: 786
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Evaluating model for company: AHL
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
Evaluating model for company: AKDSL
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Evaluating model for company: AMBL
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Evaluating model for company: CASH
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
Evaluating model for company: CYAN
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Evaluating model for company: DEL
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Evaluating model for company: DLL
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Evaluating model for company: ENGROH
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
Evaluating model for company: ESBL
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
Evaluating model for company: FCIBL
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Evaluating model for company: FCSC
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
Evaluating model for company: FDPL
32/32 ━━━━━━

C:\Users\anosha\miniconda3\Lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\anosha\miniconda3\Lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\anosha\miniconda3\Lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\anosha\miniconda3\Lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


Evaluating model for company: ARCTM
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Evaluating model for company: ASTM
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
Evaluating model for company: BILF
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
Evaluating model for company: CTM
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Evaluating model for company: CWSM
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Evaluating model for company: DFSM
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
Evaluating model for company: DINT
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Evaluating model for company: DSIL
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
Evaluating model for company: DWTM
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Evaluating model for company: GADT
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Evaluating model for company: GSPM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step


C:\Users\anosha\miniconda3\Lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\anosha\miniconda3\Lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\anosha\miniconda3\Lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\anosha\miniconda3\Lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


Evaluating model for company: HIRAT
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Evaluating model for company: IDRT
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Evaluating model for company: IDSM
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Evaluating model for company: IDYM
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
Evaluating model for company: JATM
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Evaluating model for company: JDMT
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Evaluating model for company: KOHTM
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Evaluating model for company: KOSM
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Evaluating model for company: NAGC
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Evaluating model for company: RCML
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Evaluating model for company: RUBY
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Evaluating model for company: SAIF
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
Evaluating model for company: SERT
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Evaluating model for company: 

...

Evaluating model for company: EXIDE
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
Evaluating model for company: GTYR
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Evaluating model for company: LOADS
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
Evaluating model for company: PTL
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Evaluating model for company: TBL
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Evaluating model for company: THALL
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Evaluating and plotting predictions for sector: FERTILIZER
Evaluating model for company: AGL
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
Evaluating model for company: AHCL
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Evaluating model for company: EFERT
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
Evaluating model for company: FATIMA
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
Evaluating model for company: FFC
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
Evaluating and plotting predictions for sector: AUTOMOBILE ASSEMBLER
Evaluating model for company: AGTL
31/31 ━━━━━━

C:\Users\anosha\miniconda3\Lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\anosha\miniconda3\Lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\anosha\miniconda3\Lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\anosha\miniconda3\Lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


Evaluating model for company: OML
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Evaluating model for company: PABC
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
Evaluating model for company: PSEL
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Evaluating model for company: SHFA
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Evaluating model for company: STPL
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
Evaluating model for company: TRIPF
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
Evaluating model for company: UBDL
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Evaluating model for company: UDPL
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Evaluating and plotting predictions for sector: POWER GENERATION & DISTRIBUTION
Evaluating model for company: ALTN
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Evaluating model for company: EPQL
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
Evaluating model for company: HUBC
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Evaluating model for company: KAPCO
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Evaluating model fo

C:\Users\anosha\miniconda3\Lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\anosha\miniconda3\Lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\anosha\miniconda3\Lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\anosha\miniconda3\Lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


Evaluating model for company: PRWM
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Evaluating model for company: STJT
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Evaluating model for company: YOUW
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
Evaluating and plotting predictions for sector: TOBACCO
Evaluating model for company: KHTC
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
Evaluating model for company: PAKT
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Evaluating model for company: PMPK
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Evaluating and plotting predictions for sector: OIL & GAS EXPLORATION COMPANIES
Evaluating model for company: MARI
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
Evaluating model for company: OGDC
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
Evaluating model for company: POL
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
Evaluating model for company: PPL
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Evaluating and plotting predictions for sector: LEASING COMPANIES
Evaluating model for company: PGLC
16/16 ━━━━━━━━━